In [ ]:
# from google.colab import files
# uploaded = files.upload()

# <font color='#edca82'>***Imports***</font>


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from matplotlib.ticker import ScalarFormatter  # Import ScalarFormatter
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn.preprocessing import quantile_transform
from scipy.stats import shapiro
import scipy.stats as stats
# edca82 097770 f5beb4

# <font color='#edca82'>***Lung Analysis***</font>


## <font color='#097770'>***I. Extracting Datasets***</font>

### <font color='#f5beb4'>***A. Normal Lung Dataset***</font>

In [ ]:
# Define the file path for the normal lung dataset
path_to_normal_txt = 'lusc-rsem-fpkm-tcga_paired.txt'

# Load the data from the TXT file into a DataFrame
# We assume that the data is tab-separated (sep='\t')
normal_lung_df = pd.read_csv(path_to_normal_txt, sep='\t')

# Print the shape (dimensions) of the Normal Lung Dataset
print(f'Shape of Normal Lung Dataset: {normal_lung_df.shape}\n')

# Separator line for clarity
print('----------------------------------------------\n')

# Print a sample of the dataset (the first few rows)
print(f'A sample of the dataset:\n\n {normal_lung_df.head()}')

# Now, the 'normal_lung_df' DataFrame contains the data from the specified file

Shape of Normal Lung Dataset: (19648, 52)

----------------------------------------------

A sample of the dataset:

   Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TCGA-58-8386  TCGA-22-5478  \
0    HIST3H2A           92815         62.12        130.60         33.06   
1       LIN7B           64130        185.11        283.05        119.26   
2         LXN           56925        909.17        819.30        412.00   
3      CNKSR2           22866         41.81         18.29         40.93   
4       SCML1            6322        133.36        214.27        108.14   

   TCGA-22-5472  TCGA-43-5670  TCGA-60-2709  TCGA-22-5489  TCGA-77-8007  ...  \
0         35.50         73.03         60.39         92.05         66.65  ...   
1        169.07        165.57        161.02        131.51        198.47  ...   
2        743.43       1340.84        607.87       1709.26       1709.26  ...   
3         67.12         54.72         29.27         20.26         23.76  ...   
4        109.66        190.34  

### <font color='#f5beb4'>***B. Cancerous Lung Dataset***</font>

In [ ]:
# Define file paths for the cancerous datasets
path_to_cancerous_txt = 'lusc-rsem-fpkm-tcga-t_paired.txt'

# Load the data from the TXT files into DataFrames
# We assume that the data is tab-separated (sep='\t')
cancerous_lung_df = pd.read_csv(path_to_cancerous_txt, sep='\t')

# Print the shape (dimensions) of the Cancerous Lung Dataset
print(f'Shape of Cancerous Lung Dataset: {cancerous_lung_df.shape}\n')

# Separator line for clarity
print('----------------------------------------------\n')

# Print a sample of the dataset (the first few rows)
print(f'A sample of the dataset:\n\n {cancerous_lung_df.head()}')

# Now, the 'cancerous_lung_df' DataFrame contains the data from the specified file

Shape of Cancerous Lung Dataset: (19648, 52)

----------------------------------------------

A sample of the dataset:

   Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TCGA-58-8386  TCGA-22-5478  \
0    HIST3H2A           92815        336.79        500.46        703.28   
1       LIN7B           64130        105.15        212.78        102.25   
2         LXN           56925        848.22        236.21        271.48   
3      CNKSR2           22866         32.59          8.51         45.85   
4       SCML1            6322         84.63         74.58         67.12   

   TCGA-22-5472  TCGA-43-5670  TCGA-60-2709  TCGA-22-5489  TCGA-77-8007  ...  \
0        287.01        486.75         70.51        145.02         14.03  ...   
1        212.78        172.65        244.57        105.89        152.28  ...   
2        759.08         61.25        620.67        329.84        599.49  ...   
3          6.16         49.21         11.91         12.27         15.00  ...   
4         57.89        102.9

## <font color='#097770'>***II. Calculating Difference Between Normal and Cancerous Gene Expression Levels***</font>

### <font color='#f5beb4'>***A. Concatenating Normal and Cancerous Gene Expression Levels***</font>

In [ ]:
# Get the list of all genes
allGenes_lung = normal_lung_df['Hugo_Symbol'].unique()

# Initialize a dictionary to store expression levels with gene names as keys
exprLung_data = {}

# Iterate through all genes
for gene in allGenes_lung:
    # Get the expression levels for the current gene
    normalLung_expression = normal_lung_df[normal_lung_df['Hugo_Symbol'] == gene].iloc[:, 2:].values.flatten()
    cancerousLung_expression = cancerous_lung_df[cancerous_lung_df['Hugo_Symbol'] == gene].iloc[:, 2:].values.flatten()

    # Store the expression levels in the dictionary with the gene name as the key
    exprLung_data[gene] = {
        'Normal': normalLung_expression,
        'Cancerous': cancerousLung_expression
    }

# Now exprLung_data is a dictionary where each key is a gene name, and the value is a dictionary
# containing 'Normal' and 'Cancerous' expression levels as NumPy arrays
# You can access the data for a specific gene using its name as the key

print('Printing a sample of our normal and cancerous expression levels concatenated with their corresponding gene:\n')

# Iterate through the first five genes and their expression data
for i, (gene, expression_data) in enumerate(exprLung_data.items()):
    if i >= 5:  # Stop after the first five genes
        break
    print(f'Gene: {gene}')  # Print the gene name
    print('Normal:', expression_data['Normal'])  # Print normal expression levels
    print('Cancerous:', expression_data['Cancerous'],'\n')  # Print cancerous expression levels

numValues_per_gene_lung = len(next(iter(exprLung_data.values()))['Normal'])

# Print the shape of exprLung_data
print(f'Shape of exprLung_data: {len(exprLung_data)} genes x {numValues_per_gene_lung} values per gene')

Printing a sample of our normal and cancerous expression levels concatenated with their corresponding gene:

Gene: HIST3H2A
Normal: [ 62.12 130.6   33.06  35.5   73.03  60.39  92.05  66.65  54.33  15.56
  55.49  30.34  14.45 100.83 131.51  59.55  87.03  71.5   23.08  48.87
  72.52 115.97  45.21  74.58   8.92  27.44  40.64  35.76  59.13  20.86
  37.05  48.18  51.71  49.56  72.01  21.94  27.44  35.    76.17  28.86
  90.77  59.55  40.07  22.92  29.91  82.29   4.7   37.32  43.63  77.25]
Cancerous: [ 336.79  500.46  703.28  287.01  486.75   70.51  145.02   14.03  397.93
  318.57  463.65    6.78  242.88  135.24  363.56  231.32  403.5   136.19
  130.6   285.03  156.59  954.43  454.09  110.43   16.51  447.82  103.69
  187.71  127.89  718.08  836.53   98.73   80.57  260.38  197.09  223.41
  343.89  303.44  115.16   74.06    3.06  420.68  109.66  106.63 1233.75
  172.65  303.44  228.13  251.48   23.59] 

Gene: LIN7B
Normal: [185.11 283.05 119.26 169.07 165.57 161.02 131.51 198.47 175.07 147.06
 

### <font color='#f5beb4'>***B. Handling Null Values***</font>

In [ ]:
# Initialize a list to store the indices of genes with more than 50% zeros
genes_with_more_than_50_percent_zeros_lung = []

# Iterate through all genes
for i, gene in enumerate(exprLung_data):
    # Get the normal expression levels for the current gene
    normal_expression = exprLung_data[gene]['Normal']
    cancerous_expression = exprLung_data[gene]['Cancerous']

    # Count the number of zeros for normal and cancerous expressions
    num_zeros_normal = len(normal_expression[normal_expression == 0])
    num_zeros_cancerous = len(cancerous_expression[cancerous_expression == 0])

    # Calculate the percentage of zeros for normal and cancerous expressions
    percentage_zerosNormal = (num_zeros_normal / len(normal_expression)) * 100
    percentage_zerosCancerous = (num_zeros_cancerous / len(cancerous_expression)) * 100

    # Check if the percentage of zeros is more than 50% for both normal and cancerous expressions
    if percentage_zerosNormal >= 50 or percentage_zerosCancerous >= 50:
        genes_with_more_than_50_percent_zeros_lung.append(i)

In [ ]:
# Filter out genes with more than 50% zeros from exprLung_data
allGenes_lung_filtered = [gene for i, gene in enumerate(exprLung_data) if i not in genes_with_more_than_50_percent_zeros_lung]

# Get the list of genes that were filtered out
filteredGenes_Lung = [gene for i, gene in enumerate(exprLung_data) if i in genes_with_more_than_50_percent_zeros_lung]

# Create new dictionaries to store the filtered and removed data
exprLung_data_filteredGenes = {gene: exprLung_data[gene] for gene in allGenes_lung_filtered}
exprLung_data_removedGenes = {gene: exprLung_data[gene] for gene in filteredGenes_Lung}

# Print the number of filtered and removed genes
print(f"Number of filtered genes: {len(exprLung_data_filteredGenes)}")
print(f"Number of removed genes: {len(exprLung_data_removedGenes)}","\n")

# Separator line for clarity
print('----------------------------------------------\n')

# Printing out a sample of removed genes
print('A sample of first 10 genes removed for having null values greater than 50%:\n')

# Get the list of genes in exprLung_data_removedGenes
genes_in_removedGenes = list(exprLung_data_removedGenes.keys())

# Iterate over the first ten genes in exprLung_data_removedGenes
for i, gene in enumerate(genes_in_removedGenes[:10]):
    if i >= 10:
        break
    index_in_exprLung_data = list(exprLung_data.keys()).index(gene)
    print(f"Gene Name: {gene}, Index in exprLung_data (1-based): {index_in_exprLung_data+1}")

Number of filtered genes: 17284
Number of removed genes: 2364 

----------------------------------------------

A sample of first 10 genes removed for having null values greater than 50%:

Gene Name: MAS1L, Index in exprLung_data (1-based): 18
Gene Name: CELF6, Index in exprLung_data (1-based): 54
Gene Name: RBMY1F, Index in exprLung_data (1-based): 68
Gene Name: CDH9, Index in exprLung_data (1-based): 72
Gene Name: G6PC2, Index in exprLung_data (1-based): 77
Gene Name: C1orf158, Index in exprLung_data (1-based): 85
Gene Name: GLRA1, Index in exprLung_data (1-based): 86
Gene Name: OR1N2, Index in exprLung_data (1-based): 94
Gene Name: CTD-2545G14.7, Index in exprLung_data (1-based): 101
Gene Name: CABP5, Index in exprLung_data (1-based): 109


### <font color='#f5beb4'>***C. Calculating Differences Between Normal and Cancerous Gene Expression Levels***</font>

In [ ]:
# Initialize a list to store the differences
exprLung_differences = []

# Iterate through the genes
for gene, expression_data in exprLung_data_filteredGenes.items():
    # Get the normal and cancerous expression levels
    normal_levels = expression_data['Normal']
    cancerous_levels = expression_data['Cancerous']

    # Calculate the differences for each gene
    differences = cancerous_levels - normal_levels

    # Add the differences to the list
    exprLung_differences.append((gene, differences))

# Now exprLung_differences contains the gene name and the corresponding differences for each gene
print('Printing a sample of difference between normal and cancerous expression levels concatenated with their corresponding gene:\n')

# Print the first five lists
for gene, differences in exprLung_differences[:5]:
    print(f'Gene: {gene}')  # Print the gene name
    print('Differences:', differences,'\n')  # Print the list of differences for the gene

# Check the shape of exprLung_differences
valPerGene_Lung = len(exprLung_differences[0][1])
print(f'Shape of exprLung_differences: {len(exprLung_differences)} genes x {valPerGene_Lung} values per gene')

Printing a sample of difference between normal and cancerous expression levels concatenated with their corresponding gene:

Gene: HIST3H2A
Differences: [ 274.67  369.86  670.22  251.51  413.72   10.12   52.97  -52.62  343.6
  303.01  408.16  -23.56  228.43   34.41  232.05  171.77  316.47   64.69
  107.52  236.16   84.07  838.46  408.88   35.85    7.59  420.38   63.05
  151.95   68.76  697.22  799.48   50.55   28.86  210.82  125.08  201.47
  316.45  268.44   38.99   45.2   -87.71  361.13   69.59   83.71 1203.84
   90.36  298.74  190.81  207.85  -53.66] 

Gene: LIN7B
Differences: [ -79.96  -70.27  -17.01   43.71    7.08   83.55  -25.62  -46.19   83.5
   71.73   98.51  115.22  143.76  -32.23 -301.     12.53    0.    -88.81
  -38.39   85.76  212.6   168.31  -25.95 -132.24   33.24   92.31  -32.99
  -76.21   43.18  221.28   -8.11  -25.74   88.64  125.53  -31.52   50.93
   78.1   -33.79  -15.55  -74.65  -49.87   15.98   48.25  271.68   31.08
  -73.42  -52.37   -8.47   92.09 -202.71] 

Gene: L

## <font color='#097770'>***III. Normality Test [Wilk Shapiro Test]***</font>

In [ ]:
# Initialize lists to store genes based on normality test results
normalLung_diff = []  # List to store genes with normal distribution
non_normalLung_diff = []  # List to store genes without normal distribution

# Function for Shapiro-Wilk normality test
def normality_test(data, alpha=0.05):
    """
    Performs the Shapiro-Wilk normality test on a dataset.

    Args:
        data (array-like): The dataset to be tested for normality.
        alpha (float, optional): The significance level for the test. Defaults to 0.05.

    Returns:
        None: Appends the gene and its data to normal_diff or non_normal_diff based on the test result.
    """
    #calculating the test statistics and the p value with the Shapiro-Wilk function from library
    shapiro_statistics, shapiro_p = stats.shapiro(data)

    # Seeing the significance level with the p value output from the function
    if shapiro_p > alpha:
        normalLung_diff.append((gene, data))
    elif shapiro_p < alpha:
        non_normalLung_diff.append((gene, data))

# Applying the normality test to each gene's differences
for gene, differences in exprLung_differences:
    normality_test(differences)

# Now the genes are divided into normal and non-normal based on the Shapiro-Wilk test results

# Count how many genes passed and did not pass the normality test and printing them
print(f"Number of genes that passed the normality test: {len(normalLung_diff)}")
print(f"Number of genes that did not pass the normality test: {len(non_normalLung_diff)}")


Number of genes that passed the normality test: 5629
Number of genes that did not pass the normality test: 11655


## <font color='#097770'>***IV. Identifying DEGs***</font>



### <font color='#f5beb4'>***Wilcoxon: Rank-Sum test***</font>
---
* Null Hypothesis (Ho): There is no difference in gene expression between normal and cancerous conditions.
* Alternative Hypothesis (Ha): There is a difference in gene expression between normal and cancerous conditions.

In [ ]:
# Initialize lists to store the test results
wilcoxon_significant = []
wilcoxon_not_significant = []
alpha = 0.05

def wilcoxon_signed_rank_test(data, alpha=0.05):

    """
    a function to do the signed rank wilcoxon test on the dataset.
    as after the normality test, the data was partially normal and partially non normal

    Arguments:
        data : array like structure that takes in the data that will be tested
        alpha : it's The significance level for the test. Defaults to 0.05.
    Returns:
    doesn't return a value, it just appends in the lists

    """

    # Perform the Wilcoxon signed rank test
    test_statistic, p_value = stats.wilcoxon(data)

    # Check if the p-value is less than the significance level
    if p_value < alpha:
        wilcoxon_significant.append((gene, test_statistic, p_value))
    elif p_value > alpha:
        wilcoxon_not_significant.append((gene, test_statistic, p_value))

# Applying the Wilcoxon signed rank test to each gene's differences that passed the normality test
for gene, differences in normalLung_diff:
    wilcoxon_signed_rank_test(differences)

# wilcoxon_results shall contains the gene name, test statistic, and p-value for each gene

# Check the number of significant genes after the Wilcoxon signed rank test
significant_genes = sum(1 for _, _, p_value in wilcoxon_significant if p_value < alpha)
non_significant_genes = sum(1 for _, _, p_value in wilcoxon_not_significant if p_value > alpha)
print(f"Number of significant genes: {significant_genes}")
print(f"Number of non significant genes: {non_significant_genes}")


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Number of significant genes: 4476
Number of non significant genes: 1153


## <font color='#097770'>***V. Exporting The Results***</font>

### <font color='#f5beb4'>***Wilcoxon Null Hypothesis Test[ For Non-Normality ]***</font>

In [ ]:
# Create a CSV file to store p-values
with open('WilcoxonTest_Non-normal_Lung.csv', 'w', newline='') as csvfile:
    fieldnames = ['Gene', 'p-value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for gene_name, p_value in failed_to_rejectLung_null_hypothesis:
        writer.writerow({'Gene': gene_name, 'p-value': p_value})

    for gene_name, p_value in rejectedLung_null_hypothesis:
        writer.writerow({'Gene': gene_name, 'p-value': p_value})

In [ ]:
# Create a TXT file to store p-values
with open('WilcoxonTest_Non-normal_Lung.txt', 'w') as txtfile:
    # Write headers
    txtfile.write('Gene, p-value\n')

    # Write p-values for genes that failed to reject null hypothesis
    for gene_name, p_value in failed_to_rejectLung_null_hypothesis:
        txtfile.write(f'{gene_name}, {p_value}\n')

    # Write p-values for genes that rejected null hypothesis
    for gene_name, p_value in rejectedLung_null_hypothesis:
        txtfile.write(f'{gene_name}, {p_value}\n')

# <font color='#edca82'>***Kidney Analysis***</font>

## <font color='#097770  '>***I. Extracting Datasets***</font>

### <font color='#f5beb4  '>***A. Normal Kidney Dataset***</font>

In [ ]:
# Define file paths for the normal datasets
path_to_normal_txt = 'kirc-rsem-fpkm-tcga_paired.txt'

# Load the data from the TXT files into DataFrames
# We assume that the data is tab-separated (sep='\t')
normal_kidney_df = pd.read_csv(path_to_normal_txt, sep='\t')

# Print the shape (dimensions) of the Normal Kidney Dataset
print(f'Shape of Normal Kidney Dataset: {normal_kidney_df.shape}\n')

# Separator line for clarity
print('----------------------------------------------\n')

# Print a sample of the dataset (the first few rows)
print(f'A sample of the dataset:\n\n {normal_kidney_df.head()}')

# Now, the 'normal_kidney_df' DataFrame contains the data from the specified file

Shape of Normal Kidney Dataset: (19216, 70)

----------------------------------------------

A sample of the dataset:

   Hugo_Symbol  Entrez_Gene_Id  TCGA-CJ-5680  TCGA-CZ-5453  TCGA-CW-5591  \
0    METTL21B           25895        237.86        155.50        309.83   
1      VSTM2B          342865          0.00          0.00          0.38   
2     TXNDC16           57544         92.05        199.85        171.45   
3      ZBTB49          166793         87.65         69.03         25.35   
4        SYT1            6857         27.05        138.10          0.73   

   TCGA-A3-3387  TCGA-CZ-5982  TCGA-CJ-5679  TCGA-CZ-5984  TCGA-CJ-5681  ...  \
0        629.35        426.57        309.83        267.73        371.22  ...   
1          0.00          0.91          0.38          2.48          2.53  ...   
2         82.29        108.14        128.79        119.26        120.94  ...   
3         35.76         40.07         68.55         84.63         68.07  ...   
4         12.83         27.25

### <font color='#f5beb4  '>***B. Cancerous Kidney Dataset***</font>

In [ ]:
# Define file paths for the cancerous datasets
path_to_cancerous_txt = 'kirc-rsem-fpkm-tcga-t_paired.txt'

# Load the data from the TXT files into DataFrames
# We assume that the data is tab-separated (sep='\t')
cancerous_kidney_df = pd.read_csv(path_to_cancerous_txt, sep='\t')

# Print the shape (dimensions) of the Cancerous Kidney Dataset
print(f'Shape of Cancerous Kidney Dataset: {cancerous_kidney_df.shape}\n')

# Separator line for clarity
print('----------------------------------------------\n')

# Print a sample of the dataset (the first few rows)
print(f'A sample of the dataset:\n\n {cancerous_kidney_df.head()}')

# Now, the 'cancerous_kidney_df' DataFrame contains the data from the specified file

Shape of Cancerous Kidney Dataset: (19216, 70)

----------------------------------------------

A sample of the dataset:

   Hugo_Symbol  Entrez_Gene_Id  TCGA-CJ-5680  TCGA-CZ-5453  TCGA-CW-5591  \
0    METTL21B           25895        234.57        299.25        171.45   
1      VSTM2B          342865          0.00          0.00          0.00   
2     TXNDC16           57544         57.89        182.55        122.64   
3      ZBTB49          166793         39.50         77.79         35.25   
4        SYT1            6857          3.00          0.36          7.63   

   TCGA-A3-3387  TCGA-CZ-5982  TCGA-CJ-5679  TCGA-CZ-5984  TCGA-CJ-5681  ...  \
0        285.03        265.87        463.65        348.71        195.72  ...   
1          0.73          0.00          0.00          0.39        334.46  ...   
2        100.13        189.02         69.52        133.36        129.69  ...   
3         59.55         94.67         23.42         59.13         56.68  ...   
4          3.14          0

## <font color='#097770'>***II. Calculating Difference Between Normal and Cancerous Gene Expression Levels***</font>

### <font color='#f5beb4'>***A. Concatenating Normal and Cancerous Gene Expression Levels***</font>

In [ ]:
# Get the list of all genes
allGenes_kidney = normal_kidney_df['Hugo_Symbol'].unique()

# Initialize a dictionary to store expression levels with gene names as keys
exprKidney_data = {}

# Iterate through all genes
for gene in allGenes_kidney:
    # Get the expression levels for the current gene
    normalKidney_expression = normal_kidney_df[normal_kidney_df['Hugo_Symbol'] == gene].iloc[:, 2:].values.flatten()
    cancerousKidney_expression = cancerous_kidney_df[cancerous_kidney_df['Hugo_Symbol'] == gene].iloc[:, 2:].values.flatten()

    # Store the expression levels in the dictionary with the gene name as the key
    exprKidney_data[gene] = {
        'Normal': normalKidney_expression,
        'Cancerous': cancerousKidney_expression
    }

# Now exprKidney_data is a dictionary where each key is a gene name, and the value is a dictionary
# containing 'Normal' and 'Cancerous' expression levels as NumPy arrays
# You can access the data for a specific gene using its name as the key

print('Printing a sample of our normal and cancerous expression levels concatenated with their corresponding gene:\n')

# Iterate through the first five genes and their expression data
for i, (gene, expression_data) in enumerate(exprKidney_data.items()):
    if i >= 5:  # Stop after the first five genes
        break
    print(f'Gene: {gene}')  # Print the gene name
    print('Normal:', expression_data['Normal'])  # Print normal expression levels
    print('Cancerous:', expression_data['Cancerous'],'\n')  # Print cancerous expression levels

numValues_per_gene_kidney = len(next(iter(exprKidney_data.values()))['Normal'])

# Print the shape of exprKidney_data
print(f'Shape of exprKidney_data: {len(exprKidney_data)} genes x {numValues_per_gene_kidney} values per gene')

Printing a sample of our normal and cancerous expression levels concatenated with their corresponding gene:

Gene: METTL21B
Normal: [237.86 155.5  309.83 629.35 426.57 309.83 267.73 371.22 175.07 267.73
 155.5  173.85 211.31 283.05 107.38 157.68 182.55 140.04 256.78 187.71
 104.42 285.03 314.17 346.29 275.28 136.19 325.29 156.59 124.37 166.73
 460.44 251.48 176.29 164.42 260.38 186.4  158.79 269.6   69.03 177.53
 297.17 232.94 289.02 117.6  301.33 231.32 249.73 132.44 414.87 190.34
 191.67 255.   191.67 154.42 155.5  163.28 136.19 214.27  98.73 429.54
 115.16 177.53 199.85 221.86 262.2  109.66 134.3  368.65]
Cancerous: [234.57 299.25 171.45 285.03 265.87 463.65 348.71 195.72 115.16 348.71
 149.12 100.13 293.07 228.13 281.09 480.04 150.17 110.43 211.31 255.
 122.64 295.11 379.04 275.28 325.29 177.53 279.14 244.57 172.65 329.84
 244.57 301.33 332.14 423.61 295.11 316.37 220.32 249.73 289.02 305.55
 392.44 417.77 211.31 241.19 139.07 256.78 111.99 441.64 483.38 167.9
 209.84 209.84 503.95

### <font color='#f5beb4'>***B. Handling Null Values***</font>

In [ ]:
# Initialize a list to store the indices of genes with more than 50% zeros
genes_with_more_than_50_percent_zeros_kidney = []

# Iterate through all genes
for i, gene in enumerate(exprKidney_data):
    # Get the normal expression levels for the current gene
    normal_expression = exprKidney_data[gene]['Normal']
    cancerous_expression = exprKidney_data[gene]['Cancerous']

    # Count the number of zeros for normal and cancerous expressions
    num_zeros_normal = len(normal_expression[normal_expression == 0])
    num_zeros_cancerous = len(cancerous_expression[cancerous_expression == 0])

    # Calculate the percentage of zeros for normal and cancerous expressions
    percentage_zerosNormal = (num_zeros_normal / len(normal_expression)) * 100
    percentage_zerosCancerous = (num_zeros_cancerous / len(cancerous_expression)) * 100

    # Check if the percentage of zeros is more than 50% for both normal and cancerous expressions
    if percentage_zerosNormal >= 50 or percentage_zerosCancerous >= 50:
        genes_with_more_than_50_percent_zeros_kidney.append(i)

In [ ]:
# Filter out genes with more than 50% zeros from exprKidney_data
allGenes_kidney_filtered = [gene for i, gene in enumerate(exprKidney_data) if i not in genes_with_more_than_50_percent_zeros_kidney]

# Get the list of genes that were filtered out
filteredGenes_Kidney = [gene for i, gene in enumerate(exprKidney_data) if i in genes_with_more_than_50_percent_zeros_kidney]

# Create new dictionaries to store the filtered and removed data
exprKidney_data_filteredGenes = {gene: exprKidney_data[gene] for gene in allGenes_kidney_filtered}
exprKidney_data_removedGenes = {gene: exprKidney_data[gene] for gene in filteredGenes_Kidney}

# Print the number of filtered and removed genes
print(f"Number of filtered genes: {len(exprKidney_data_filteredGenes)}")
print(f"Number of removed genes: {len(exprKidney_data_removedGenes)}","\n")

# Separator line for clarity
print('----------------------------------------------\n')

# Printing out a sample of removed genes
print('A sample of first 10 genes removed for having null values greater than 50%:\n')

# Get the list of genes in exprKidney_data_removedGenes
genes_in_removedGenes = list(exprKidney_data_removedGenes.keys())

# Iterate over the first ten genes in exprKidney_data_removedGenes
for i, gene in enumerate(genes_in_removedGenes[:10]):
    if i >= 10:
        break
    index_in_exprKidney_data = list(exprKidney_data.keys()).index(gene)
    print(f"Gene Name: {gene}, Index in exprKidney_data (1-based): {index_in_exprKidney_data+1}")


Number of filtered genes: 17034
Number of removed genes: 2182 

----------------------------------------------

A sample of first 10 genes removed for having null values greater than 50%:

Gene Name: VSTM2B, Index in exprKidney_data (1-based): 2
Gene Name: MRGPRX2, Index in exprKidney_data (1-based): 10
Gene Name: INSL5, Index in exprKidney_data (1-based): 19
Gene Name: RP11-15E18.4, Index in exprKidney_data (1-based): 23
Gene Name: C3orf22, Index in exprKidney_data (1-based): 30
Gene Name: CXorf28, Index in exprKidney_data (1-based): 32
Gene Name: RP11-865B13.1, Index in exprKidney_data (1-based): 33
Gene Name: AADACL2, Index in exprKidney_data (1-based): 34
Gene Name: KIAA1658, Index in exprKidney_data (1-based): 37
Gene Name: ALPPL2, Index in exprKidney_data (1-based): 50


### <font color='#f5beb4'>***C. Calculating Differences Between Normal and Cancerous Gene Expression Levels***</font>

In [ ]:
# Initialize a list to store the differences
exprKidney_differences = []

# Iterate through the genes
for gene, expression_data in exprKidney_data_filteredGenes.items():
    # Get the normal and cancerous expression levels
    normal_levels = expression_data['Normal']
    cancerous_levels = expression_data['Cancerous']

    # Calculate the differences for each gene
    differences = cancerous_levels - normal_levels

    # Add the differences to the list
    exprKidney_differences.append((gene, differences))

# Now exprKidney_differences contains the gene name and the corresponding differences for each gene

print('Printing a sample of difference between normal and cancerous expression levels concatenated with their corresponding gene:\n')

# Print the first five lists
for gene, differences in exprKidney_differences[:5]:
    print(f'Gene: {gene}')  # Print the gene name
    print('Differences:', differences,'\n')  # Print the list of differences for the gene

# Check the shape of exprKidney_differences
valPerGene_kidney = len(exprKidney_differences[0][1])

print(f'Shape of exprKidney_differences: {len(exprKidney_differences)} genes x {valPerGene_kidney} values per gene')

Printing a sample of difference between normal and cancerous expression levels concatenated with their corresponding gene:

Gene: METTL21B
Differences: [  -3.29  143.75 -138.38 -344.32 -160.7   153.82   80.98 -175.5   -59.91
   80.98   -6.38  -73.72   81.76  -54.92  173.71  322.36  -32.38  -29.61
  -45.47   67.29   18.22   10.08   64.87  -71.01   50.01   41.34  -46.15
   87.98   48.28  163.11 -215.87   49.85  155.85  259.19   34.73  129.97
   61.53  -19.87  219.99  128.02   95.27  184.83  -77.71  123.59 -162.26
   25.46 -137.74  309.2    68.51  -22.44   18.17  -45.16  312.28  199.17
   95.98   69.66  343.85   62.93  132.59 -184.97  436.4   316.03  161.19
  178.85  182.52   31.36  186.5   -10.11] 

Gene: TXNDC16
Differences: [ -34.16  -17.3   -48.81   17.84   80.88  -59.27   14.1     8.75  -82.73
   86.62  -81.17 -173.28   14.5   -51.06   21.28   62.62  -38.12  -54.3
  -47.91    9.06  -70.33   22.27   -9.8    47.97   66.47   -1.09   88.17
  -94.36  -46.9    65.07  -25.62    4.51   60.17

## <font color='#097770'>***III. Normality Test [Wilk Shapiro Test]***</font>

In [ ]:
# Initialize lists to store genes based on normality test results
normalKidney_diff = []  # List to store genes with normal distribution
non_normalKidney_diff = []  # List to store genes without normal distribution

# Function for Shapiro-Wilk normality test
def normality_test(data, alpha=0.05):
    """
    Performs the Shapiro-Wilk normality test on a dataset.

    Args:
        data (array-like): The dataset to be tested for normality.
        alpha (float, optional): The significance level for the test. Defaults to 0.05.

    Returns:
        None: Appends the gene and its data to normal_diff or non_normal_diff based on the test result.
    """
    #calculating the test statistics and the p value with the Shapiro-Wilk function from library
    shapiro_statistics, shapiro_p = stats.shapiro(data)

    # Seeing the significance level with the p value output from the function
    if shapiro_p > alpha:
        normalKidney_diff.append((gene, data))
    elif shapiro_p < alpha:
        non_normalKidney_diff.append((gene, data))

# Applying the normality test to each gene's differences
for gene, differences in exprKidney_differences:
    normality_test(differences)

# Now the genes are divided into normal and non-normal based on the Shapiro-Wilk test results

# Count how many genes passed and did not pass the normality test and printing them
print(f"Number of genes that passed the normality test: {len(normalKidney_diff)}")
print(f"Number of genes that did not pass the normality test: {len(non_normalKidney_diff)}")

Number of genes that passed the normality test: 6504
Number of genes that did not pass the normality test: 10530


## <font color='#097770'>***IV. Identifying DEGs***</font>




### <font color='#f5beb4'>***Wilcoxon: Rank-Sum test***</font>
---
* Null Hypothesis (Ho): There is no difference in gene expression between normal and cancerous conditions.
* Alternative Hypothesis (Ha): There is a difference in gene expression between normal and cancerous conditions.

In [ ]:
# Initialize lists to store the test results
wilcoxon_significant = []
wilcoxon_not_significant = []
alpha = 0.05

def wilcoxon_signed_rank_test(data, alpha=0.05):

    """
    a function to do the signed rank wilcoxon test on the dataset.
    as after the normality test, the data was partially normal and partially non normal

    Arguments:
        data : array like structure that takes in the data that will be tested
        alpha : it's The significance level for the test. Defaults to 0.05.
    Returns:
     doesn't return a value, it just appends in the lists


    """

    # Perform the Wilcoxon signed rank test
    test_statistic, p_value = stats.wilcoxon(data)

    # Check if the p-value is less than the significance level
    if p_value < alpha:
        wilcoxon_significant.append((gene, test_statistic, p_value))
    elif p_value > alpha:
        wilcoxon_not_significant.append((gene, test_statistic, p_value))

# Applying the Wilcoxon signed rank test to each gene's differences that passed the normality test
for gene, differences in exprKidney_differences:
    wilcoxon_signed_rank_test(differences)

# wilcoxon_results shall contains the gene name, test statistic, and p-value for each gene

# Check the number of significant genes after the Wilcoxon signed rank test
significant_genes = sum(1 for _, _, p_value in wilcoxon_significant if p_value < alpha)
non_significant_genes = sum(1 for _, _, p_value in wilcoxon_not_significant if p_value > alpha)
print(f"Number of significant genes: {significant_genes}")
print(f"Number of non significant genes: {non_significant_genes}")


Number of significant genes: 13240
Number of non significant genes: 3794


## <font color='#097770'>***V. Exporting The Results***</font>

### <font color='#f5beb4'>***Wilcoxon Null Hypothesis Test[ For Non-Normality ]***</font>

In [ ]:
# Create a CSV file to store p-values
with open('WilcoxonTest_Non-normal_Kidney.csv', 'w', newline='') as csvfile:
    fieldnames = ['Gene', 'p-value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for gene_name, p_value in failed_to_rejectKidney_null_hypothesis:
        writer.writerow({'Gene': gene_name, 'p-value': p_value})

    for gene_name, p_value in rejectedKidney_null_hypothesis:
        writer.writerow({'Gene': gene_name, 'p-value': p_value})

In [ ]:
# Create a TXT file to store p-values
with open('WilcoxonTest_Non-normal_Kidney.txt', 'w') as txtfile:
    # Write headers
    txtfile.write('Gene, p-value\n')

    # Write p-values for genes that failed to reject null hypothesis
    for gene_name, p_value in failed_to_rejectKidney_null_hypothesis:
        txtfile.write(f'{gene_name}, {p_value}\n')

    # Write p-values for genes that rejected null hypothesis
    for gene_name, p_value in rejectedKidney_null_hypothesis:
        txtfile.write(f'{gene_name}, {p_value}\n')

### <font color='#f5beb4'>***T-test Null Hypothesis Test [ For Normality ]***</font>

In [ ]:
# Create a CSV file to store p-values and results
with open('T-test_Normal_Kidney.csv', 'w', newline='') as csvfile:
    fieldnames = ['Gene', 'T-Statistic', 'P-Value', 'Result']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for gene_name, t_statistic, p_value, result in gene_resultsKidney:
        writer.writerow({'Gene': gene_name, 'T-Statistic': t_statistic, 'P-Value': p_value, 'Result': result})


# Create a text file to store p-values and results
with open('gene_results.txt', 'w') as file:
    file.write("Gene Results:\n")
    for gene_name, t_statistic, p_value, result in gene_resultsKidney:
        file.write(f"Gene: {gene_name}\n")
        file.write(f"T-Statistic: {t_statistic:.4f}\n")
        file.write(f"P-Value: {p_value:.4f}\n")
        file.write(f"Result: {result}\n\n")

In [ ]:
# Create a text file to store p-values and results
with open('T-test_Normal_Kidney.txt', 'w') as file:
    file.write("Gene Results:\n")
    for gene_name, t_statistic, p_value, result in gene_resultsKidney:
        file.write(f"Gene: {gene_name}\n")
        file.write(f"T-Statistic: {t_statistic:.4f}\n")
        file.write(f"P-Value: {p_value:.4f}\n")
        file.write(f"Result: {result}\n\n")